In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import warnings
warnings.simplefilter('ignore')
from classes.model_pipeline import ClassificationPipeline, RegressionPipeline
from classes.utils import gather_data_from_folders
from classes.text_features import TextFeatureExtractor
from dotenv import load_dotenv
from tqdm.auto import tqdm
import os
from pandas.api.types import is_numeric_dtype
import pandas as pd

load_dotenv();

In [6]:
# playlists_dir = 'audio_sentiment_classifier/playlists'
# playlists_dir = os.path.join("/".join(os.path.abspath(os.path.realpath(__name__)).split("/")[:-1]), "playlists")
df = pd.read_csv("final_data.csv")#gather_data_from_folders(playlists_dir)

In [7]:
df.shape

(5375, 571)

# Training

In [23]:
target = 'acousticness'

In [24]:
cat_features = [ f for f in df.select_dtypes(include=['category']).columns if f != target ]
num_features = [ f for f in df.select_dtypes(include=['number']).columns if f != target ]

In [25]:
# model = ClassificationPipeline(df, target_column=target, num_features=num_features, cat_features=cat_features)
model = RegressionPipeline(df, target_column=target, num_features=num_features, cat_features=cat_features)
model.get_regression_pipeline()

INFO: Initializing pipeline for target variable: acousticness
INFO: Setting up the pipeline...
INFO: Setting up the pipeline...


TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('num',
                                                                                       Pipeline(steps=[('imputer',
                                                                                                        SimpleImputer()),
                                                                                                       ('scaler',
                                                                                                        StandardScaler())]),
                                                                                       ['popularity',
                                                                                        'album_release_year',
                                                                                        'duration_ms',
                                                                                        'danceability',
                                                                                        'energy',
                                                                                        'key',
                                                                                        'loudness',
                                                                                        'mode',
                                                                                        'speechiness',
                                                                                        'instrumentalness',
                                                                                        'liveness',
                                                                                        'valence',
                                                                                        'tempo',
                                                                                        'time_sig...
                                                                                        'mfcc_6',
                                                                                        'mfcc_7',
                                                                                        'mfcc_8',
                                                                                        'mfcc_9',
                                                                                        'mfcc_10',
                                                                                        'mfcc_11',
                                                                                        'mfcc_12',
                                                                                        'mfcc_13',
                                                                                        'chroma_1',
                                                                                        'chroma_2',
                                                                                        'chroma_3', ...]),
                                                                                      ('cat',
                                                                                       Pipeline(steps=[('imputer',
                                                                                                        SimpleImputer(strategy='most_frequent')),
                                                                                                       ('onehot',
                                                                                                        OneHotEncoder(handle_unknown='ignore'))]),
                                                                                       [])])),
                                                     ('model',
                                                      RandomForestRegressor

In [26]:
%%time
model.run_pipeline(f'dashboard/results/{target}')

Pipeline Progress:   0%|          | 0/5 [00:00<?, ?step/s]

INFO: Starting pipeline execution for prediction of acousticness...
INFO: Splitting the data into training and testing sets...
INFO: Data splitting completed.
INFO: Training the model...
INFO: Model training completed.
INFO: Evaluating regression model...
INFO: Regression evaluation completed.
INFO: Performing SHAP analysis for regression...


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [17]:
model.evaluate_model()

INFO: Evaluating classification model...
INFO: Classification evaluation completed.


,Metric,Score
0,Accuracy,0.734884
1,Precision,0.720870
2,Recall,0.734884
3,F1 Score,0.718155


# Automated

In [99]:
features_to_exclude = ['title', 'artist', 'mp3_path', 'lyrics']

In [100]:
features_to_predict = ['explicit',
       'album_release_year', 'duration_ms',
        'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'tempo_extracted', 'zcr']

In [115]:
df['explicit'] = df['explicit'].map({True: 1, False: 0})

In [116]:
for target in tqdm(features_to_predict):
    if os.path.exists(f"dashboard/results/{target}"):
        continue
    cat_features = [ f for f in df.select_dtypes(include=['category']).columns if f != target and f not in features_to_exclude]
    num_features = [ f for f in df.select_dtypes(include=['number']).columns if f != target and f not in features_to_exclude]

    if is_numeric_dtype(df[target]):
        model = RegressionPipeline(df, target_column=target, num_features=num_features, cat_features=cat_features)
    else:
        model = ClassificationPipeline(df, target_column=target, num_features=num_features, cat_features=cat_features)
        
    model.run_pipeline(f'dashboard/results/{target}')

  0%|          | 0/17 [00:00<?, ?it/s]

INFO: Initializing pipeline for target variable: explicit
INFO: Setting up the pipeline...


Pipeline Progress:   0%|          | 0/5 [00:00<?, ?step/s]

INFO: Starting pipeline execution...
INFO: Splitting the data into training and testing sets...
INFO: Data splitting completed.
INFO: Fixing class balance...
INFO: Applied SMOTE with k_neighbors=5 to fix class imbalance.
INFO: Model training completed.
INFO: Evaluating classification model...
INFO: Classification evaluation completed.
INFO: Performing SHAP analysis for classification...


 98%|===================| 2346/2400 [00:45<00:01]        

...

INFO: SHAP analysis for regression completed.
INFO: Saving results to dashboard/results/album_release_year...
INFO: Evaluating regression model...
INFO: Regression evaluation completed.
INFO: Evaluation results saved to dashboard/results/album_release_year/evaluation_results.csv
INFO: SHAP summary plot saved to dashboard/results/album_release_year/shap_summary_plot.png
INFO: SHAP beeswarm plot saved to dashboard/results/album_release_year/shap_beeswarm_plot.png
INFO: Pipeline execution completed.
INFO: Initializing pipeline for target variable: duration_ms
INFO: Setting up the pipeline...


Pipeline Progress:   0%|          | 0/5 [00:00<?, ?step/s]

INFO: Starting pipeline execution...
INFO: Splitting the data into training and testing sets...
INFO: Data splitting completed.
INFO: Training the model...
INFO: Model training completed.
INFO: Evaluating regression model...
INFO: Regression evaluation completed.
INFO: Performing SHAP analysis for regression...


 99%|===================| 1187/1200 [01:18<00:00]        

INFO: SHAP analysis for regression completed.
INFO: Saving results to dashboard/results/duration_ms...
INFO: Evaluating regression model...
INFO: Regression evaluation completed.
INFO: Evaluation results saved to dashboard/results/duration_ms/evaluation_results.csv
INFO: SHAP summary plot saved to dashboard/results/duration_ms/shap_summary_plot.png
INFO: SHAP beeswarm plot saved to dashboard/results/duration_ms/shap_beeswarm_plot.png
INFO: Pipeline execution completed.
